In [ ]:
import casadi as ca
import casadi.tools

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

# Inverted pendulum CasADi example

The dynamics are described by 4 states;

- Position of cart, pos [m]
- Angle of pendulum, theta [rad]
- Derivative of cart position, dpos [m/s]
- Derivative of pendulum angle, dtheta [rad/s]

As input we have force acting on the cart, F [N]

In [ ]:
# Physical constants
g = 9.82    # gravitation [m/s^2]
L = 0.2     # pendulum length [m]
m = 1       # pendulum mass [kg]
m_cart = 0.5 # cart mass [kg]

In [ ]:
# State vector
x = ca.tools.struct_symMX([
    'p',
    'theta',
    'dp',
    'dtheta'
])

# Input 'vector'
u = ca.tools.struct_symMX(['F'])

# Sum of forces
F_tot = u['F'] + m*L*x['dtheta']*ca.sin(x['theta']) - m * g * ca.sin(x['theta']) * ca.cos(x['theta'])

# Sum of masses
m_tot = m_cart + m - m * ca.cos(x['theta'])**2

# d(dp)/dt
ddpos = F_tot / m_tot
# d(dtheta)/dt
ddtheta = g/L*ca.sin(x['theta']) - ca.cos(x['theta'])*ddpos

# ODE Right-hand side
rhs = ca.vertcat(x['dp'], x['dtheta'], ddpos, ddtheta)

# Our dynamics
f = ca.Function('f', [x, u], [rhs], ['x', 'u'], ['dx/dt'])

f

# Integration

Now we have dynamics described through CasADi symbols and functions.
We can now introduce integrators for e.g. simulation.

Here we introduce an integrator to simulate a single sample -- this is to formulate discrete dynamics from our continous dynamics.

So we move from our formulation of:

$$
\dot{x} = f(t,x,u)
$$

to:

$$
x[k+1] = F(x[k], u[k])
$$

where $F$ is the integration of $f$ over one sample period of $\Delta t$.

The integration can be done in many different ways; CasADi includes an interface to common integrators from *SUNDIALS*, but it is also easy (and sometimes very beneficial) to built e.g. a RK4 integrator, which minimizes additional overhead.

In [ ]:
dt = 0.02 # [s], 100 Hz sampling

# Reference Runge-Kutta implementation simulating exactly one sample
RK4 = ca.integrator('RK4', 'rk', {'x': x, 'p': u, 'ode': f(x,u)}, {'number_of_finite_elements': 1, 'tf': dt})

# Discretized (sampling time dt) system dynamics as a CasADi Function
F_RK4 = ca.Function('F_RK4', [x, u], [RK4(x0=x, p=u)["xf"]], ['x[k]', 'u[k]'], ['x[k+1]'])

# RK4
k1 = f(x, u)
k2 = f(x + dt / 2.0 * k1, u)
k3 = f(x + dt / 2.0 * k2, u)
k4 = f(x + dt * k3, u)
xf = x + dt / 6.0 * (k1 + 2 * k2 + 2 * k3 + k4)

# Single step time propagation
F_RK4 = ca.Function("F_RK4", [x, u], [xf], ['x[k]', 'u[k]'], ['x[k+1]'])

# Optimal control formulations

There are two ways of posing optimization problems in CasADi; either 'bare-metal' using pure CasADi symbolics or with a higher-level interface which significantly eases problem formulation + adds helpful debugging features.

In [ ]:
opti = ca.Opti()

# Optimization horizon
N = 200 # 1 [s]

# Decision variables for states and inputs
X = opti.variable(x.size, N+1)

p = X[0,:]
dp = X[2,:]
theta = X[1,:]
dtheta = X[3,:]

U = opti.variable(N, u.size)

# Initial state is a parameter
x0 = opti.parameter(x.size)

Now we have some decision variables; we then set up constraints.
Formulating a multiple-shooting problem we pose some gap-closing constraints using our discrete dynamics.

In [ ]:
# Gap-closing shooting constraints
for k in range(N):
   opti.subject_to(X[:,k+1] == F_RK4(X[:,k], U[k]))

In [ ]:
# Path constraints
opti.subject_to(opti.bounded(-3,  p, 3)) # Limits on how far out the cart can move to either side, +- 3 m
opti.subject_to(opti.bounded(-1.2, U, 1.2)) # Limits force input on cart, +- 1.2 N

In [ ]:
# Initial and terminal constraints
opti.subject_to(X[:,0] == x0)
opti.subject_to(X[:,-1] == ca.vertcat(0,0,0,0)) # End at (0,0,0,0)

Formulate objective, here we minimize sum of squares of our input

In [ ]:
opti.minimize(ca.sumsqr(U))

Then, we setup solver specifics -- here we use *IPOPT*.

In [ ]:
#opti.solver('ipopt', {'ipopt': {'print_level': 0}})

options = {'qpsol': 'qrqp', 'expand': True}
options['qpsol_options'] = {'print_iter': False, "print_header": False}
options['print_iteration'] = False
options['print_status'] = False
opti.solver('sqpmethod', options)

And we can now set up initial conditions and solve our problem:

In [ ]:
opti.set_value(x0, [0.5, 0, 0, 0])
sol = opti.solve()

x_traj = sol.value(X).T[:-1]
u_traj = sol.value(U).reshape(1,-1).T

sol_traj = pd.DataFrame(np.hstack((x_traj, u_traj)), columns=['pos', 'theta', 'dpos', 'dtheta', 'F'])

In [ ]:
sol_traj[['F', 'pos']].plot()

# MPC

In [ ]:
OCP = opti.to_function("OCP", [x0], [U[0]], ["x0"], ["u"])

In [ ]:
OCP([0.5, 0, 0, 0])

In [ ]:
sol_traj["F"][0]

In [ ]:
N_sim = 400
x_traj = np.zeros((N_sim, x.size))
u_traj = np.zeros((N_sim, u.size))

x_traj[0,:] = [0.5, 0, 0, 0]
for i in range(1, N_sim):
    u_traj[i, :] = OCP(x_traj[i-1, :])
    
    # Simulate one sample
    x_traj[i, :] = F_RK4(x_traj[i-1,:], u_traj[i]).toarray().flatten()

In [ ]:
plt.spy(sol.value(ca.hessian(opti.f,opti.x)[0]))

In [ ]:
sim_res = pd.DataFrame(np.hstack((x_traj, u_traj)), columns=['pos', 'theta', 'dpos', 'dtheta', 'F'])

In [ ]:
sim_res[['pos']].plot()

# Extras

In [ ]:
OCP.save("ocp.casadi")

In [ ]:
OCP_loaded = ca.Function.load("ocp.casadi")

In [ ]:
OCP_loaded

In [ ]:
OCP.generate("ocp_codegen", {"main": True})